In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
import cv2
import numpy as np
import os
from skimage.measure import shannon_entropy

def is_salt_and_pepper_image(image):
  ## Aflam dimensiunile imaginii
  h, w = image.shape

  ## Returnează coordonatele (rânduri, coloane) pentru toți pixelii cu valoarea 255. np.where
  ## Combina coordonatele în perechi de forma (row, col), pentru a avea o listă de poziții. np.column_stack
  white_pixel_positions = np.column_stack(np.where(image == 255))

  # Obținem coordonatele pixelilor cu valoarea 0 (negru)
  black_pixel_positions = np.column_stack(np.where(image == 0))

  # Combinam vectorii cu coordonatele pixelilor
  all_noisy_positions = np.vstack((white_pixel_positions, black_pixel_positions))
  # print(all_noisy_positions[:50, :])

  outliers_threshold = 100

  outliers = []

  for pos in all_noisy_positions:
    # Coordonatele pixelului
    x, y = pos

    # Extragem vecinătatea (ferestra 3x3)
    x_min = max(x - 1, 0)
    x_max = min(x + 2, image.shape[0])
    y_min = max(y - 1, 0)
    y_max = min(y + 2, image.shape[1])

  # Valori vecini (excludem pixelul central)
    neighborhood = image[x_min:x_max, y_min:y_max]

    # Excludem pixelul central
    neighbors = np.delete(neighborhood.flatten(), (x - x_min) * (y_max - y_min) + (y - y_min))


    # Calculăm media și deviația standard a vecinilor
    mean = np.mean(neighbors)
    std = np.std(neighbors)

    # Verificăm dacă valoarea pixelului se încadrează în interval
    if not (mean - 2 * std <= image[x, y] <= mean + 2 * std):
        outliers.append(pos)  # Adăugăm pixelul la lista de outlieri

  print(len(outliers))
  return len(outliers) > outliers_threshold



folder_path = "gray_images"
image_files = [f for f in os.listdir(folder_path) if f.endswith(".jpg") or f.endswith(".png")]

noisy_images = []

for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if is_salt_and_pepper_image(image):
        noisy_images.append(image_file)

print("Imagini cu zgomot sare și piper:", noisy_images)


178
5
5
181
Imagini cu zgomot sare și piper: ['truck_noisy.png', 'noisy_palace.png']


In [ ]:
## Antrenare
## [imagine_HR, imagine_LR]
## 3 parti
## Generatorul in acest process de antrenare. Imi genereaza imagini HR din imaginea de rezolutie mica imagine_LR
## Discrimiatorul in acest process de antrenare 'iti evalueaza' cat de realista este imaginea data de generator in raport cu cea data ca input (referinta)
## Se compara iterativ imaginea generata cu cea reala -> diverse metode de comparatie ca analogie ganditiva la o functie de similaritate